In [47]:
!pip install pandas
!pip install numpy
import requests
import json
import pandas
import numpy

DEFAULTAPIKEY='ac83034cfa742c0f79c26e9a612b4ba7e2aa0d3d'

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [22]:
# tinkering to work out the typeform api

In [23]:
def get_all_typeforms(apikey=DEFAULTAPIKEY):
    '''
    get all typeforms available with the api key provided
    
    params
        apikey    - string. default: hardcoded nastiness
    
    returns
        list of dicts. eg: {'id':'SoMeID', 'name': 'Some Name'}
    '''
    url = "https://api.typeform.com/v1/forms?key={}".format(apikey)
    return requests.get(url).json()

get_all_typeforms()


[{'id': 'PfSNFM', 'name': 'Keboola Test Dungeon'}]

In [43]:
def get_form_data(form_id=None, apikey=DEFAULTAPIKEY):
    '''
    get a python object representing all of the info available 
    for a particular form.
    
    params
        form_id   - string. default: first form returned 
                    by get_all_typeforms()
        apikey    - string. default: hardcoded nastiness
    
    '''
    form_id = get_all_typeforms()[0]['id'] if not form_id else form_id
    url = "https://api.typeform.com/v1/form/{}?key={}".format(form_id, apikey)
    
    return requests.get(url).json()

get_form_data()


{'http_status': 200,
 'questions': [{'field_id': 46098446,
   'id': 'date_46098446',
   'question': 'This is a date field. Say "Hello, date field!" So, what date is today?'},
  {'field_id': 46099137,
   'id': 'number_46099137',
   'question': 'This is a number field. It contains\xa0numbers. What is your favorite prime number? #GeekAllert'},
  {'field_id': 46099317,
   'id': 'yesno_46099317',
   'question': 'We went rogue and added the checkmark field! Is Keboola the best company in the known universe to work for?'},
  {'field_id': 46100308,
   'id': 'list_46100308_choice',
   'question': 'Now is time for the multi-choice question! What is your favorite kind of Pho?'},
  {'field_id': 46100513,
   'id': 'textarea_46100513',
   'question': 'The dirtiest data field of all data fields. The address field. The true nightmare of every data analyst!'}],
 'responses': [{'answers': {'date_46098446': '2017-03-17',
    'list_46100308_choice': 'I love all kinds of Pho. Pho for the president!',
    '

In [63]:
def get_values_or_nan(data, keys):
    return [data[k] if k in data.keys() else numpy.NaN for k in keys]

def get_responses_csv(form_id=None, apikey=DEFAULTAPIKEY):
    '''
    returns form responses as a CSV
    '''
    # load and checks
    data = get_form_data(form_id, apikey)
    if 'responses' not in data.keys() or len(data['responses']) < 1:
        raise ValueError('no responses found, aborting.')
   
    # fetch column names
    # assumption: all metadata objs are created with the same fields
    metadata_fields = list(data['responses'][0]['metadata'].keys())
    question_ids = [i['id'] for i in data['questions']]
    question_texts = [i['question'] for i in data['questions']]
    
    # init table
    cols = ['user_token'] + question_texts + metadata_fields
    table = pandas.DataFrame(columns=cols)
        
    for response in data['responses']:
        # add user token column data
        row = [response['token']]
        
        # add response answers
        row += get_values_or_nan(response['answers'], question_ids)

        # add response metadata
        row += get_values_or_nan(response['metadata'], metadata_fields)
        
        table = table.append(pandas.DataFrame([row], columns=cols))
        
    return table.set_index('user_token').to_csv()

get_responses_csv()

,"This is a date field. Say ""Hello, date field!"" So, what date is today?",This is a number field. It contains numbers. What is your favorite prime number? #GeekAllert,We went rogue and added the checkmark field! Is Keboola the best company in the known universe to work for?,Now is time for the multi-choice question! What is your favorite kind of Pho?,The dirtiest data field of all data fields. The address field. The true nightmare of every data analyst!,user_agent,referer,network_id,platform,date_land,date_submit,browser
user_token,,,,,,,,,,,,
7ee7cb68882c57d3c81b7503e8ec4157,2017-03-17,3,1,I love all kinds of Pho. Pho for the president!,"210 - 1461 Johnston Rd, White Rock, BC V4B 3Z4",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:22:34,2017-03-17 21:23:50,default
929fb7f0fc2c011d98d49fd807e3ba12,2016-12-31,7,NaN,"More ingredients, the better. I like my Pho fu...",15230 Russell Ave White Rock BC V4B,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:24:06,2017-03-17 21:25:08,default
c141209e02e6a9bf01c60ece7169b55b,NaN,NaN,NaN,NaN,NaN,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:25:14,0000-00-00 00:00:00,default
bc92448137641bd58b9296061089514e,1921-04-01,13,1,I don't eat Pho but I'll start immediately! So...,"Krizikova 488/115, 186 00 Praha 8-Karlin, Czechia",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:25:46,2017-03-17 21:27:06,default
a3b36c47a8fe8b59fb574fdebaba71ac,2007-01-01,10,1,Beef and basic is my choice of Pho.,"30 Cecil St, #19-06 Singapore 049712",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:27:15,2017-03-17 21:30:07,default
7a66caab65c92d6ffdfb1d8cea7ded26,2015-02-16,17,1,NaN,"WeWork 9 Devonshire Square, London EC2M 4YD, U...",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:31:22,2017-03-17 21:32:34,default
25d7e68aee7db37ce8d282bb969942cc,NaN,101,NaN,I love all kinds of Pho. Pho for the president!,"1150 Davie St, Vancouver, BC V6E 1N1",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,https://hrkeboola.typeform.com/to/PfSNFM,783db4e9d8,other,2017-03-17 21:32:43,2017-03-17 21:34:04,default
4fc901e2410e7d5ccc8c228e0ba858fe,2017-03-23,7,1,Beef and basic is my choice of Pho.,"123 Evergreen Terrace\nSpringfield, IL 12345",Mozilla/5.0 (X11; Fedora; Linux x86_64) AppleW...,https://inhumantsar.typeform.com/to/PfSNFM,e367c5e34e,other,2017-03-23 19:35:28,2017-03-23 19:37:34,default
4e25bb5a8fc1b27dd11e86dba040a815,2017-03-23,1,NaN,NaN,"Left the multi-choice blank, just because",Mozilla/5.0 (X11; Fedora; Linux x86_64) AppleW...,https://inhumantsar.typeform.com/to/PfSNFM,e367c5e34e,other,2017-03-23 19:37:45,2017-03-23 19:38:23,default


nan